In [138]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# my own imports
import sklearn.model_selection as skl

In this homework your task will be to modify `DecisionTreeClassifier` class from your practice in class.

### Task 1 (0.5 points) <br/>

Download Telecom Data To Analyse The Churn Data Set `telecom_churn.csv`. Read it using `pandas.read_csv()` function. To open the function description use `Shift` + `Tab` . Show the first 5 rows of the dataset using `pandas.DataFrame.head()` function.

[Dataset Information](https://www.kaggle.com/spscientist/telecom-data/download):

Columns:
* state (State letter code)
* account length (How long the client has been served by the company)
* area code (Phone number prefix)
* phone number
* international plan (International roaming (connected / not connected))
* voice mail plan (Voice mail (connected / not connected))
* number vmail messages (Number of voice messages)
* total day minutes (Total duration of conversations during the day)
* total day calls (Total calls during the day)
* total day charge (Total amount of payment for services during the day)
* total eve minutes (Total duration of conversations in the evening)
* total eve calls (Total number of calls in the evening)
* total eve charge (Total amount of payment for services in the evening)
* total night minutes (Total duration of conversations at night)
* total night calls (Total number of calls at night)
* total night charge (Total amount of payment for services at night)
* total intl minutes (Total duration of international calls)
* total intl calls (Total number of international call)
* total intl charge (Total payment for international calls)
* customer service calls (The number of calls to the service center)
* churn

Churn is target: True - client has left.

In [139]:
data = pd.read_csv('telecom_churn.csv')
data.head()

,state,account length,area code,phone number,international plan,voice mail plan,number vmail messages,total day minutes,total day calls,total day charge,...,total eve calls,total eve charge,total night minutes,total night calls,total night charge,total intl minutes,total intl calls,total intl charge,customer service calls,churn
0,KS,128,415,382-4657,no,yes,25,265.1,110,45.07,...,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,371-7191,no,yes,26,161.6,123,27.47,...,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,358-1921,no,no,0,243.4,114,41.38,...,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,375-9999,yes,no,0,299.4,71,50.90,...,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,330-6626,yes,no,0,166.7,113,28.34,...,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


* Which features are categorical? 
* Which features can be considered as object ID? Should we keep them? Why?

In [140]:
'''
Categorial variables:
    - state
    - international plan
    - voice mail plan
    - churn
'''

'''
I see two possibilities for storing object-ID: making it by ourselves
like it is done here - rowId = objectId
or saving phone-number as objectID. Other variables couldnt be taken as objectID because
they are not unique for every single object.

However I think there is no reason to keep phone-number in data because
it is just making tree more complex but it doesnt bring any valuable data
'''

'\nI see two possibilities for storing object-ID: making it by ourselves\nlike it is done here - rowId = objectId\nor saving phone-number as objectID. Other variables couldnt be taken as objectID because\nthey are not unique for every single object.\n\nHowever I think there is no reason to keep phone-number in data because\nit is just making tree more complex but it doesnt bring any valuable data\n'

### Task 2 (0.5 points) <br/>

The target column for classification is `Churn`. However, it is categorical feature, so you need to encode this by `0` and `1` values (False = 0, True = 1). Implement this mapping.

In [141]:
# data['churn'] selects all elements in column with name 'churn'
# map is evident 
data['churn'] = data['churn'].map({True: 1, False: 0})
data.head()

,state,account length,area code,phone number,international plan,voice mail plan,number vmail messages,total day minutes,total day calls,total day charge,...,total eve calls,total eve charge,total night minutes,total night calls,total night charge,total intl minutes,total intl calls,total intl charge,customer service calls,churn
0,KS,128,415,382-4657,no,yes,25,265.1,110,45.07,...,99,16.78,244.7,91,11.01,10.0,3,2.70,1,0
1,OH,107,415,371-7191,no,yes,26,161.6,123,27.47,...,103,16.62,254.4,103,11.45,13.7,3,3.70,1,0
2,NJ,137,415,358-1921,no,no,0,243.4,114,41.38,...,110,10.30,162.6,104,7.32,12.2,5,3.29,0,0
3,OH,84,408,375-9999,yes,no,0,299.4,71,50.90,...,88,5.26,196.9,89,8.86,6.6,7,1.78,2,0
4,OK,75,415,330-6626,yes,no,0,166.7,113,28.34,...,122,12.61,186.9,121,8.41,10.1,3,2.73,3,0


Create matrix `X` and vector of labels `y`. Split them into train and test samples using `sklearn.model_selection.train_test_split()` function from scikit-learn library. Also, set up random state in the function `random_state=42`.

Expected output dataframes names: `df_X_train`, `df_X_test` и `y_train`, `y_test`

In [142]:
# making a vector and matrix from dataframe
y = data['churn']
X = data.loc[:, data.columns != 'churn']

# splitting using function
df_X_train, df_X_test, y_train, y_test = skl.train_test_split(X, y, test_size=0.2, random_state=42)

In [143]:
# Check your result
assert(round(df_X_test.size/X.size, 1)==0.2)
assert(round(y_test.size/y.size, 1)==0.2)

### Task 3 (2 points) categorical feature encoding

Use (0, 1) for binary features. Replace values both for `df_X_train` and `df_X_test`

In [144]:
'''
These categorial features are changed
    international plan
    voice mail plan
'''

def replace_to_binary(df):
    df['international plan'] = df['international plan'].map({"no": 0, "yes": 1})
    df['voice mail plan'] = df['voice mail plan'].map({"no": 0, "yes": 1})
    
replace_to_binary(df_X_train)
replace_to_binary(df_X_test)

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


Now, let's implement target encoding for other categorical features. For this task we propose you to implement the simpliest form of target encoding, which goes as following:

Training:
* Consider categorical feature $f$
* For each category $c$ in this feature calculate mean value of target feature $y$: $v_c$

Application:
* Consider feature $f$
* Replace each category $c$ with $v_c$
* If $v_c$ is not calculated (possible new category) - replace $c$ with global target mean

Create the next funсtions:
* `learn_target_encoding `
    * Input: train dataframe, target array, list of features names for encoding
    * Output: nested dict with mapping from category to target encoded value for each categorical feature, global target mean value
* `apply_target_encoding`
    * Input: dataframe, encoding_dict
    * Output: transformed dataframe in form of numpy array

HINT for pandas:
* `df.groupby(..)`
* `df.column.to_dict(..)`
* `df.column.replace(some_dict)`

In [343]:
def learn_target_encoding(df, y, features2encode):    
    encoding_dict = {}
    float_type = type(0.0)
    
    for feature2encode in features2encode:
        for row_value in df[feature2encode]:
            # no need to check if dictionary already contains key
            if (row_value not in encoding_dict):
                # we need to encode this row_value based on y
                # firstly finding all elements (their indeces) with same row_value
                indeces_with_value = df.index[df[feature2encode] == row_value].tolist()
                
                # then mean of same indices in y
                mean_feature = np.mean([y[i] for i in indeces_with_value])
                
                # insert that value to dictionary
                pair = {row_value : mean_feature}
                encoding_dict.update(pair)
    
    target_mean = np.mean(y)
    
    return encoding_dict, target_mean

def apply_target_encoding(df_in, encoding_dict, global_target_mean):    
    df_out = df_in.copy(True)
    
    for i in range(len(df_in.columns)):             
        
        # not changing column if type of keys in dictionary is not the same
        # as in dataframe column
        if (type(df_out.iloc[:, i][0]) != type(list(encoding_dict.keys())[0])):
            continue
        
        # replacing with values from dictionary
        # and then replacing NaNs that were created on previous step
        df_out[df_in.columns[i]] = df_out[df_in.columns[i]].map(encoding_dict)
        df_out[df_in.columns[i]] = df_out[df_in.columns[i]].replace(np.NaN, global_target_mean)
        
    return df_out.to_numpy()

In [344]:
# Check your functions:
input_df = pd.read_csv("task4_train_only_for_check.csv", sep="\t")
features_list = ['cat2']

X_dataframe = input_df[features_list]
y_array = input_df['y']
test_dataframe = pd.read_csv("task4_test_only_for_check.csv", sep="\t")

dictionary, glob_mean = learn_target_encoding(X_dataframe, y_array, features_list)
output_matrix = apply_target_encoding(test_dataframe, dictionary, glob_mean)

assert(np.array_equal(output_matrix, np.array([[0, 0, 0, 1, 1, 0], [3, 3, 5, 4, 2, 3.076923076923077]]).T))

### Task 4 (1 point) <br/>

Now let's look at this data. For each input feature plot historgrams, as it was done in you practice in class. How do you think, what features are the most informative? 

In [ ]:
# Your code here

### Task 5 (1 point) <br/>

Fit `DecisionTreeClassifier` from you practice in class with this sample. Find the best parameters. What is `accuracy` of the classification on the test sample?  

In [ ]:
# Your code here

### Task 6 (3 points) <br/>

Implement feature importance estimation in `DecisionTreeClassifier`. Importance of a feature $f$ is defined as follows:

* Let $T(f)$ be the set of all nodes, relying on feature $f$ when making split.
* Efficiency of split at node $t$: $\Delta I(t)=I(t)-\sum_{c\in childen(t)}\frac{n_{c}}{n_{t}}I(c)$,  where $n_t$, $n_c$ is number of samples in nodes t, c
* Feature importance of $f$: $\sum_{t\in T(f)}n_{t}\Delta I(t)$, where $n_t$ is number of samples in node t

Calculate importance of input features in your dataset. What features are the most important (informative) for the classification?

To do this you have to update your decision_tree learning procedure:
* Return best_information_gain from best_split
* Save best_information_gain in node
* Traverse tree recursively and caclulate feature importance for every feature.

In [ ]:
# Your code here
# Output: array 'features_order' from highest to lowest importance (first 5)

In [ ]:
assert(len(features_order)==5)
assert(np.array_equal(features_order, np.array(['total day charge', 'customer service calls', 'international plan', 'total intl calls', 'total eve charge'])))

### Task 7 (2 points) <br/>

Implement Reduced Error Pruning in you `DecisionTreeClassifier`. 

Fit the classifier similar to **Task 6** setting up `max_depth=20`. 

Prune this decision tree using train data. 

Create a plot "Accuracy (y-axis) vs Numbers of vertices (x-axis)".

In [ ]:
# Your code here